In [ ]:
import requests
import time 
import datetime
import csv

In [ ]:
#Вставьте свой токен
access_token = ''
owner_id = '-215426617' #1tsprint vk id

### Функция получения ответа API vk.com в виде Json

In [ ]:
def getjson(url, data = None):
    response = requests.get(url, params = data)
    response.raise_for_status()
    print(response.url, '\n')
    return response.json()

### Функция получения всех записей на стене сообщества и их общего количества, метод wall.get

In [ ]:
def get_all_posts (access_token, owner_id, count = 100, offset=0):
    all_posts = []
     
    while True:
        time.sleep(1)
        wall = getjson("https://api.vk.com/method/wall.get", {
            'owner_id' : owner_id, 
            'count' : count,
            'access_token' : access_token,
            'offset' : offset,
            'v' : '5.92'
            })
         
        count_posts = wall['response']['count']
        posts = wall['response']['items']
         
        all_posts.extend(posts)
         
        if len(all_posts) >= count_posts:
            break
        else:
            offset += 100
            
    return all_posts, count_posts

### Функция обработки ответа API vk.com (полученных методом wall.get)

In [ ]:
def make_posts(all_posts):
    filtered_data = []
    for post in all_posts:
         
        try:
            id = post['id']
        except:
            id = 0
             
        try:
            owner_id = str(post['owner_id'])[1:]
        except:
            owner_id = ''
             
        try:
            link = 'https://vk.com/wall-{owner_id}_{id}'.format(owner_id = owner_id, id = id)
        except:
            link = ''
             
        try:
            date = datetime.datetime.fromtimestamp(int(post['date'])).strftime('%d-%m-%Y')
        except:
            date = ''
         
        try:
            likes = post['likes']['count']
        except:
            likes = 0
             
        try:
            reposts = post['reposts']['count']
        except:
            reposts = 0
             
        try:
            comments = post['comments']['count']
        except:
            comments = 0
             
        try:
            text = post['text']
        except:
            text = ''
             
        filtered_post = {
                'id' : id,
                'date' : date,
                'likes' : likes,
                'reposts' : reposts,
                'comments' : comments,
                'text' : str(text),
                'link' : link,
                'group_id' : owner_id,
                }
         
        filtered_data.append(filtered_post)
     
    return filtered_data

### Извлечение данных из ответа API vk.com  (полученных методом wall.get)

In [ ]:
all_posts, count_posts = get_all_posts (access_token, owner_id)
ref_posts = make_posts(all_posts)
count_reposts = sum([dic['reposts'] for dic in ref_posts])

### Получение количества подписчиков, , метод groups.getById

In [ ]:
wall = requests.get("https://api.vk.com/method/groups.getById", {
        'group_ids' : owner_id[1:],
        'grour_id' : owner_id[1:],
        'access_token' : access_token,
        'fields' : 'members_count',
        'v' : '5.131'
        })
wall.raise_for_status()
count_users = wall.json()['response'][0]['members_count']

### Запись данных задания в файл

In [ ]:
answer = list()
answer.append({'Показатель': 'Количество подписчиков', 'Значение':count_users})
answer.append({'Показатель': 'Количество постов', 'Значение':count_posts})
answer.append({'Показатель': 'Количество репостов', 'Значение':count_reposts})                   

with open('task_3.2.csv', 'w', newline='') as f:
    fieldnames = ['Показатель', 'Значение']
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(answer)